In [1]:
import datetime
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

# OpenAI function with LangChain

In [2]:
import json

# Example dummy function hard coded to return the same weather
# In production, this could be your backend API or an external API
def get_current_weather(location, date, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "-4",
        "unit": unit,
        "date": date,
        "forecast": ["snow", "windy"],
    }
    return json.dumps(weather_info)

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from pydantic.v1 import BaseModel, Field
from enum import Enum
from datetime import date

llm = ChatOpenAI(model="gpt-4-0613", temperature=0)

class TemperatureScale(str, Enum):
    CELSIUS = "celsius"
    FAHRENHEIT = "fahrenheit"

class WeatherInput(BaseModel):
    location: str = Field(description="The city")
    date: str = Field( description="The date")
    unit: TemperatureScale = Field(default=TemperatureScale.CELSIUS)


tools = [
    StructuredTool(
        name="get_current_weather",
        func=get_current_weather,
        description="Get the current weather in a given city at a given date",
        args_schema=WeatherInput
    )
]

agent_executor = initialize_agent(
    tools, 
    llm, 
    agent=AgentType.OPENAI_FUNCTIONS, 
    verbose=True, 
    max_execution_time=30
)

In [4]:
agent_executor.invoke("What's your suggestion for holiday activity in Munich at 2nd December 2023?")



> Entering new AgentExecutor chain...

Invoking: `get_current_weather` with `{'location': 'Munich', 'date': '2023-12-02'}`


{"location": "Munich", "temperature": "-4", "unit": "celsius", "date": "2023-12-02", "forecast": ["snow", "windy"]}Given the weather forecast for Munich on 2nd December 2023, it's going to be quite cold with a temperature of -4 degrees Celsius and there will be snow and wind. 

Here are some indoor activities you might enjoy:

1. **Visit Museums**: Munich is home to many world-class museums. You could visit the Deutsches Museum, which is the world's largest museum of science and technology, or the Munich Residenz, which is the former royal palace of the Wittelsbach monarchs of Bavaria.

2. **Enjoy Bavarian Food**: You could visit a traditional Bavarian restaurant or beer hall, such as the famous Hofbräuhaus, to enjoy some hearty local cuisine.

3. **Shopping**: Munich has several large shopping centers, such as the Olympia Einkaufszentrum, where you could do so

{'input': "What's your suggestion for holiday activity in Munich at 2nd December 2023?",
 'output': "Given the weather forecast for Munich on 2nd December 2023, it's going to be quite cold with a temperature of -4 degrees Celsius and there will be snow and wind. \n\nHere are some indoor activities you might enjoy:\n\n1. **Visit Museums**: Munich is home to many world-class museums. You could visit the Deutsches Museum, which is the world's largest museum of science and technology, or the Munich Residenz, which is the former royal palace of the Wittelsbach monarchs of Bavaria.\n\n2. **Enjoy Bavarian Food**: You could visit a traditional Bavarian restaurant or beer hall, such as the famous Hofbräuhaus, to enjoy some hearty local cuisine.\n\n3. **Shopping**: Munich has several large shopping centers, such as the Olympia Einkaufszentrum, where you could do some holiday shopping.\n\n4. **Spa Day**: Consider a relaxing day at one of Munich's many spas, such as the Müller'sches Volksbad, whic